# Python tips

This page gathers some tips that can be useful when coding in Python.


### Dynamic attributes with setattr / getattr

We can set or get an attribute with a name specified dynamically using `setattr()` and `getattr()` functions.

In [4]:
class Person:
    pass

person = Person()

# set/get an attribute with a fixed name
person.attr1 = 1
print('Fixed attribute =', person.attr1)

# set/get an attribute with a dynamic name
setattr(person, 'attr2', 2)
print('Dynamic attribute =', getattr(person, 'attr2'))

Fixed attribute = 1
Dynamic attribute = 2


### Password input

When a script requires to enter a password, using the built-in `input()` function is not good, because it shows the password on the screen.   
Instead we can use the `getpass()` function that will hide the password.

In [9]:
from getpass import getpass

# using input() : the password appears in clear
pwd = input('Password : ')
print('Login with password : ', pwd)

# using getpass() : the password appears in clear
pwd = getpass('Password : ')
print('Login with password : ', pwd)

Password : abcd
Login with password :  abcd
Password : ········
Login with password :  abcd


### Getter / Setter / Deleter for Python class

Sometimes, a field of a Python class needs to be calculated from the value of other fields.  
If we just set it in the constructor, it will not be updated when one of the dependant fields is updated.  
To solve this, we can define the field as a method with the `@property` decorator so it can be accessed without brackets.

We can also define a setter and a deleter for these property fields with the `@<fieldname>.setter` and `@<fieldname>.deleter` decorators to perform an action on the dependant fields when the field is set or deleted.

In [14]:
class Person:
    def __init__(self, firstname, lastname):
        self.firstname = firstname
        self.lastname = lastname
    
    @property
    def fullname(self):
        return f'{self.firstname} {self.lastname}'
    
    @fullname.setter
    def fullname(self, val):
        self.firstname, self.lastname = val.split(' ')
        
    @fullname.deleter
    def fullname(self):
        self.firstname, self.lastname = None, None

    def __repr__(self):
        return f'Person(firstname={self.firstname}, lastname={self.lastname}, fullname={self.fullname})'
    

# Getter
person = Person('John', 'Smith')
print(person)

# Setter
person.fullname = 'Jack Black'
print(person)

# Deleter
del person.fullname
print(person)

Person(firstname=John, lastname=Smith, fullname=John Smith)
Person(firstname=Jack, lastname=Black, fullname=Jack Black)
Person(firstname=None, lastname=None, fullname=None None)


### File Manipulation



In [22]:
# Simple approach : open the file, use it and close it
f = open('test.txt', 'w')
f.write('Hello,\nWorld!')
f.close()

# Better approach using a context manager closing the file automatically
with open('test.txt', 'r') as f:
    content = f.read()

# Read file content
with open('test.txt', 'r') as f:
    content = f.read()                # entire file as a string
    first_10_chars = f.read(10)       # read the next N characters
    line = f.readline()               # read the next line
    lines = f.readlines()             # list of all lines
    for line in f :                   # iterate on all lines
        pass

# Navigate in the file
with open('test.txt', 'r') as f:
    line1 = f.readline()
    f.seek(0)                         # Move the file cursor to a given position
    line1_again = f.readline()


### Running Python system modules

Running a custom python module is done with the command : `python3 module.py`

Sometimes we need to run a system module that we did not write ourselves, this is done with `python3 -m <module>`.

A few examples are :
- Starting a local mail server : `python3 -m smtpd -c DebuggingServer -n localhost:1025`
- Creating a virtual environment : `python3 -m venv proj_folder/venv`
- Running the unit tests : `python3 -m unittest -v`

We can get detailled information on the arguments these modules can take by importing them in a Python interactive shell and call `help()` :  
`python3`  
`$> import smtpd`  
`$> help(smtpd)`  

### Add a cache to a function

We can use decorator `lru_cache(maxsize=128)` (least-recently used cache) from the `functools` package.  
It will keep in cache the results of the function for the most used parameters (up to `maxsize` elements).

The decorator `@cache` is a shortcut for `@lru_cache(maxsize=None)`, that will store all results of the function.


In [28]:
from time import time
from functools import lru_cache, cache


def fibo_1(n):
    """Naive Fibonacci function with no cache"""
    return 1 if n < 3 else fibo_1(n-1) + fibo_1(n-2)

@lru_cache(maxsize=10)
def fibo_2(n):
    """Fibonacci function caching up to 10 results"""
    return 1 if n < 3 else fibo_2(n-1) + fibo_2(n-2)

@cache
def fibo_3(n):
    """Fibonacci function caching all results"""
    return 1 if n < 3 else fibo_3(n-1) + fibo_3(n-2)


print('fibo(36)')

start = time()
res = fibo_1(36)
print('No cache         :', '{:.5f} sec'.format(time() - start))

start = time()
res = fibo_2(36)
print('Cache (size 10)  :', '{:.5f} sec'.format(time() - start))

start = time()
res = fibo_3(36)
print('Cache (infinite) :', '{:.5f} sec'.format(time() - start))


print('\nfibo(1500)')

start = time()
res = fibo_2(1500)
print('Cache (size 10)  :', '{:.5f} sec'.format(time() - start))

start = time()
res = fibo_3(1500)
print('Cache (infinite) :', '{:.5f} sec'.format(time() - start))

fibo(36)
No cache         : 3.43586 sec
Cache (size 10)  : 0.00009 sec
Cache (infinite) : 0.00007 sec

fibo(1000)
Cache (size 10)  : 0.00103 sec
Cache (infinite) : 0.00074 sec


### Double-ended queues

The `collections` package contains a `deque` class (double-ended queue) that represents doubly linked lists.  
It performs O(1) read/insert/delete operations on both ends of the deque.

It can be used to implement a queue (FIFO) or a stack (LIFO).

In [8]:
from collections import deque

# FIFO
queue = deque(['Tom', 'Mary', 'Sara'])
queue.append('James')      # Add a person at the rear of the queue
tom = queue.popleft()      # Delete a person at the front of the queue

# LIFO
stack = deque(['layer1', 'layer2'])
stack.append('layer3')
layer3 = stack.pop()

# Peek
stack[0]    # peek first elem
stack[-1]   # peek last elem

'layer2'

### Priority queue (or min-heap)

A priority queue (or "min-heap") is a data structure that always allows to return the min element.  
The `heapq` module offers a priority queue mechanism in Python.

Unlike `deque`, it does not have a specific class for a priority queue.  
Instead it uses a normal list and applies push/pop functions on it via the `heapq` module.  
A min-heap is a complete binary tree (all levels are full except the last one).  
It is represented by `heapq` by a list with successive levels :
<pre>
       10
     /    \
    6     12       [10, 6, 12, 4, 7, 11, 15, 2, 5]
   / \    / \
  4   7  11  15
 / \
2   5
</pre>

A priority queue can contain either a single item (used as priority) or a (priority, object) pair.

In [23]:
import heapq

# a min-heap is defined as a list
heap = []

# insertion is made via heappush
heapq.heappush(heap, (10, 'Tom'))
heapq.heappush(heap, (5, 'Sara'))
heapq.heappush(heap, (1, 'Jim'))
heapq.heappush(heap, (8, 'Nick'))

# we can pop the smallest element with heappop
# it will restructure the list to keep the priority queue structure
print('heappop :', heapq.heappop(heap))
print('heappop :', heapq.heappop(heap))

# We can turn a list into a heap with heapify
friends = ['Tom', 'Sara', 'Jim', 'Nick']
heapq.heapify(friends)                           # binary tree list representation
print('heappop name:', heapq.heappop(friends))
print('heappop name:', heapq.heappop(friends))

# Push and pop in a single function
nbs = [5, 3, 2, 1]
heapq.heapify(nbs)
print('heappushpop:', heapq.heappushpop(nbs, 0))     # Push then pop
print('heapreplace:', heapq.heapreplace(nbs, 0))     # Pop then push

# Get the N smallest/largest items
print('nsmallest:', heapq.nsmallest(3, nbs))
print('nlargest:', heapq.nlargest(3, nbs))

heappop : (1, 'Jim')
heappop : (5, 'Sara')
heappop name: Jim
heappop name: Nick
heappushpop: 0
heapreplace: 1
nsmallest: [0, 2, 3]
nlargest: [5, 3, 2]
